In [1]:
import os
import re
import warnings

import pandas as pd
import numpy as np
import torch

from transformers import (
    AutoConfig, AutoTokenizer, AutoModelForSeq2SeqLM, 
    Seq2SeqTrainingArguments, Seq2SeqTrainer, 
    DataCollatorForSeq2Seq, 
    EarlyStoppingCallback)

from datasets import load_metric, Dataset

import wandb
import nltk

os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings('ignore')
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
NGPU = torch.cuda.device_count()
NCPU = os.cpu_count()

# Paths and Names

In [3]:
### paths and names

PROJECT_NAME = 'text_summarization'
RUN_ID = 'v1'

DATA_PATH = 'data/original/train.csv'

MODEL_CHECKPOINT = 'ainize/kobart-news'
METRIC_NAME = 'rouge'

NOTEBOOK_NAME = './train.ipynb'

ROOT_PATH = './'
SAVE_PATH = os.path.join(ROOT_PATH, 'training_results', RUN_ID)

model_name = re.sub(r'[/-]', r'_', MODEL_CHECKPOINT).lower()
run_name = f'{model_name}_{RUN_ID}'
output_dir = os.path.join(SAVE_PATH, run_name)

!mkdir -p {SAVE_PATH}

In [4]:
%env WANDB_PROJECT={PROJECT_NAME}
%env WANDB_NOTEBOOK_NAME={NOTEBOOK_PATH}
%env WANDB_LOG_MODEL=true
%env WANDB_WATCH=all
wandb.login()

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find {NOTEBOOK_PATH}.


env: WANDB_PROJECT=text_summarization
env: WANDB_NOTEBOOK_NAME={NOTEBOOK_PATH}
env: WANDB_LOG_MODEL=true
env: WANDB_WATCH=all


wandb: Currently logged in as: dotsnangles. Use `wandb login --relogin` to force relogin


True

# Training Args

In [5]:
report_to="wandb"

num_train_epochs = 15
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1

optim = 'adamw_torch' # 'adamw_torch' or 'adamw_hf'

learning_rate = 3e-6 * NGPU
weight_decay = 0.01
adam_epsilon = 1e-8

lr_scheduler_type = 'cosine' # 'linear', 'cosine', 'cosine_with_restarts', 'polynomial', 'constant', 'constant_with_warmup'
warmup_ratio = 0

save_total_limit = 2

load_best_model_at_end = True
metric_for_best_model = 'eval_loss'

save_strategy = "epoch"
evaluation_strategy = "epoch"

logging_strategy = "steps"
logging_first_step = True 
logging_steps = int(500 / NGPU)

predict_with_generate=True
generation_max_length=512
# generation_num_beams=5

fp16 = False

# Model & Tokenizer & Metric

In [6]:
config = AutoConfig.from_pretrained(MODEL_CHECKPOINT)
del config.label2id
del config.id2label
del config.task_specific_params

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_CHECKPOINT, config=config)
tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
metric = load_metric(METRIC_NAME)

# Functions

In [8]:
prefix = ""

max_input_length = 1026
max_target_length = 514

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["context"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [9]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

# Inputs and Labels

In [10]:
data_df = pd.read_csv(DATA_PATH)

In [11]:
dataset = Dataset.from_pandas(data_df).shuffle(seed=100).train_test_split(0.1)
train_dataset = dataset['train']
eval_dataset = dataset['test']

In [12]:
train_dataset = train_dataset.map(preprocess_function, 
                                  batched=True, 
                                  num_proc=NCPU, 
                                  remove_columns=train_dataset.column_names)

eval_dataset = eval_dataset.map(preprocess_function, 
                                batched=True, 
                                num_proc=NCPU, 
                                remove_columns=eval_dataset.column_names)
print(train_dataset)
print(eval_dataset)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

...

#20:   0%|          | 0/1 [00:00<?, ?ba/s]

#21:   0%|          | 0/1 [00:00<?, ?ba/s]

#22:   0%|          | 0/1 [00:00<?, ?ba/s]

#23:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 2694
})
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 300
})


# Train

In [13]:
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    run_name=run_name,
    report_to=report_to,

    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,

    optim=optim,

    learning_rate=learning_rate,
    weight_decay=weight_decay,
    adam_epsilon=adam_epsilon,

    lr_scheduler_type=lr_scheduler_type,
    warmup_ratio=warmup_ratio,

    save_total_limit=save_total_limit,

    load_best_model_at_end=load_best_model_at_end,
    metric_for_best_model=metric_for_best_model,

    save_strategy=save_strategy,
    evaluation_strategy=evaluation_strategy,

    logging_strategy=logging_strategy,
    logging_first_step=logging_first_step, 
    logging_steps=logging_steps,

    predict_with_generate=predict_with_generate,
    generation_max_length=generation_max_length,
    # generation_num_beams=generation_num_beams,

    fp16=fp16,
)

In [14]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)

trainer = Seq2SeqTrainer(
    model=model,
    
    args=training_args,
    
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    
    tokenizer=tokenizer,
    data_collator=data_collator,
    
    compute_metrics=compute_metrics,
)

In [15]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 2694
  Num Epochs = 15
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2535
  Number of trainable parameters = 123859968
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,3.338300,0.866478,57.318900,37.477800,57.006200,56.952200,39.676700
2,3.338300,0.783967,62.119300,39.007800,62.132900,61.853700,42.080000
3,0.923200,0.761290,63.749800,40.302500,63.134400,63.484200,42.076700
4,0.923200,0.747950,65.189300,41.468900,64.844000,64.967300,40.453300
5,0.923200,0.753505,63.074200,41.192200,62.687400,62.854100,42.363300
6,0.521000,0.768608,63.267800,41.078800,62.584000,63.052200,41.026700
7,0.521000,0.774921,64.824300,41.656300,64.691600,64.853200,41.926700
8,0.521000,0.776698,64.739800,42.141200,64.377300,64.819400,41.883300
9,0.382200,0.798989,65.037300,42.456900,64.552900,65.089000,42.603300
10,0.382200,0.806406,64.905800,41.937800,64.257000,64.893500,41.683300


The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 300
  Batch size = 16
Saving model checkpoint to ./training_results/v1/ainize_kobart_news_v1/checkpoint-169
Configuration saved in ./training_results/v1/ainize_kobart_news_v1/checkpoint-169/config.json
Model weights saved in ./training_results/v1/ainize_kobart_news_v1/checkpoint-169/pytorch_model.bin
tokenizer config file saved in ./training_results/v1/ainize_kobart_news_v1/checkpoint-169/tokenizer_config.json
Special tokens file saved in ./training_results/v1/ainize_kobart_news_v1/checkpoint-169/special_tokens_map.json
Deleting older checkpoint [training_results/v1/ainize_kobart_news_v1/checkpoint-676] due to args.save_total_limit
The following columns in the evaluat

TrainOutput(global_step=2535, training_loss=0.48162019661897737, metrics={'train_runtime': 3788.8139, 'train_samples_per_second': 10.666, 'train_steps_per_second': 0.669, 'total_flos': 2.452273219049472e+16, 'train_loss': 0.48162019661897737, 'epoch': 15.0})

In [16]:
keep = [
    'added_tokens.json',
    'config.json',
    'pytorch_model.bin',
    'special_tokens_map.json',
    'tokenizer.json',
    'tokenizer_config.json',
    'vocab.txt'
]

ckpts = os.listdir(output_dir)
for ckpt in ckpts:
    ckpt = os.path.join(output_dir, ckpt)
    for item in os.listdir(ckpt):
        if item not in keep:
            os.remove(os.path.join(ckpt, item))

# Generate

In [17]:
preds = trainer.predict(eval_dataset)

The following columns in the test set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 300
  Batch size = 16


In [18]:
for data, pred in zip(eval_dataset, preds.predictions):
    context = tokenizer.decode(data['input_ids'], skip_special_tokens=True)
    summary = tokenizer.decode(data['labels'], skip_special_tokens=True)
    pred = tokenizer.decode(pred[2:], skip_special_tokens=True)
    # print(f'입력: {context}')
    print(f'정답: {summary}')
    print(f'예측: {pred}', end='\n\n')

정답: 제137회 제2차 정례회 제1차 본회의 휴회가 의사일정으로 가결됨.
예측: 제117회 제2차 정례회 제1차 본회의 휴회의 건은 행정사무감사특별위원회 활동에 따른 감사계획서 작성 등 위원회 활동을 하는 의사일정으로 갈음함.

정답: 제68회 완주군의회 임시회 회의록 서명의원으로 이희창 의원과 안성근 의원이 선임됨.
예측: 제68회 완주군의회 임시회 회의록 서명의원으로 이희창 의원과 안성근 의원이 선임됨.

정답: 1997년도공유재산관리계획승인 신청안이 가결됨.
예측: 1997년도 공유재산 관리계획 승인 신청안이 가결됨.

정답: 음성군의회 사무기구 사무분장 규칙 일부개정규칙안은 상위법이 변경, 신설됨에 따라 음성군의회 전문위원의 담당 소관 실과소를 정비하기 위하여 제정되었다. 해당 안건은 가결됨.
예측: 음성군의회 사무기구 사무분장 규칙 일부개정규칙안은 <음성군 행정기구 설치 조례>와 같은 조례 시행규칙이 개정되어 실과소 명칭이 변경, 신설됨에 따라 음성군의회 전문위원의 담당 소관 실과소를 정비하기 위해 <음성군의회 사무기구 사무분장 규칙> 일부를 개정하고자 제정함. 해당 안건은 가결됨.

정답: 토목, 건축사업 용역 시 설계를 하기 전에 담당자들과 충분히 업무 연찬을 해줄 것. 특별회계에서 일반회계로 전출했던 예산을 단기간에 특별회계로 다시 전출할 수 있는 계획을 수립할 것. AI발생으로 연기된 반기문전국마라톤대회 자체감사를 시행한 후 의회에 정확한 보고를 할 것. 체육회 총회와 총회 전 이사회에서 어떤 안건이 다루어졌는지 파악할 것.
예측: 2013년도 행정사무감사 시정, 건의사항 조치결과에 대한 보고.

정답: 주요시설 비교견학 결과보고서가 채택됨.
예측: 2019년도 주요시설 비교견학 결과보고서가 채택됨.

정답: 제243회 음성군의회 임시회 회의록 서명의원으로 이대웅 의원과 김순옥 의원이 선출됨.
예측: 제243회 음성군의회 임시회 회의록 서명의원으로 이대웅 의원과 김순옥 의원이 선출됨.

정답: 제120회 음성군의회 임시회 회기는 7월 24일부